# Memory Layout, Views, and Copies

Understanding **how NumPy stores and shares data** is key for writing efficient,
bug-free code.


#### Arrays are just views on memory

A NumPy array is a **block of memory** plus:
- `shape` — dimensions
- `dtype` — data type
- `strides` — how many bytes to jump to reach the next element along each axis

In [43]:
import numpy as np

x = np.arange(9)
X = x.reshape(3, 3)
print("X.shape:  ", X.shape)
print("X.dtype:  ", X.dtype)
print("X.strides:", X.strides)

# The array is stored in memory as a contiguous block of elements:
print("\nX.flags:")
print(X.flags)

print("x.base:", x.base)
print("\nX.base:", X.base)


X.shape:   (3, 3)
X.dtype:   int64
X.strides: (24, 8)

X.flags:
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

x.base: None

X.base: [0 1 2 3 4 5 6 7 8]


### Slicing creates views

In [44]:
Y = X[:, :2]

Y[0, 0] = 999

print("Y[0, 0] =", Y[0, 0])
print("X[0, 0] =", X[0, 0])  # 999 → changed in both!

print("\nY.base is X:", Y.base is X)

Y[0, 0] = 999
X[0, 0] = 999

Y.base is X: False


In [45]:
Z = X.T          # transpose → usually a view (shared data)
W = X.T.copy()   # forces a copy

print("\nZ.base is x:", Z.base is x)
print("W.base is x:", W.base is x)
print("W.base is None:", W.base is None)

W[0, 0] = -1
print("X[0, 0] =", X[0, 0])
print("W[0, 0] =", W[0, 0])


Z.base is x: True
W.base is x: False
W.base is None: True
X[0, 0] = 999
W[0, 0] = -1


## C vs Fortran order

C and Fortran have different conventions for storing multi-dimensional arrays in memory.

- C: row-major order (rows are contiguous in memory)
- Fortran: column-major order (columns are contiguous in memory)

NumPy uses C order by default.

In [46]:
C = np.arange(6).reshape(2, 3)
print(C)

F = np.asfortranarray(C)
print(F)

print("\nC.flags:\n", C.flags)
print("F.flags:\n", F.flags)

[[0 1 2]
 [3 4 5]]
[[0 1 2]
 [3 4 5]]

C.flags:
   C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

F.flags:
   C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



In [47]:
X = np.arange(1e8).reshape(10000, 10000)

# Row-wise (fast for C-contiguous)
%timeit X.sum(axis=1)

# Column-wise (slow for C-contiguous)
%timeit X.sum(axis=0)

58.1 ms ± 1.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
55 ms ± 1.84 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
